In [1]:
import numpy as np
import pandas as pd

In [2]:
def weekly(exchange, tidm):
    '''Weekly closing prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
period = 48  # Look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly closing prices.
df = weekly(exchange, tidm)

In [5]:
# Donchian channels.
df['upr'] = df.high.rolling(period).max().shift(periods=1)
df['lwr'] = df.low.rolling(period).min().shift(periods=1)
df['mid'] = 0.5 * (df.upr + df.lwr)

In [6]:
# Trade entry & exit signals.
df['buy'] = np.where(df.close > df.upr, 1, 0)
df['sell'] = np.where(df.close < df.mid, 1, 0)
df['state'] = 0
for i in range(period, len(df)):
    if df.loc[df.index[i], 'buy'] == 1 and df.loc[df.index[i - 1], 'state'] == 0:
        df.loc[df.index[i], 'state'] = 1
    elif df.loc[df.index[i], 'sell'] == 1:
        df.loc[df.index[i], 'state'] = 0
    else:
        df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
df['entry'] = np.where(np.logical_and(df.state == 1, df.state.shift(periods=1) == 0), 1, 0)
df['exit'] = np.where(np.logical_and(df.state == 0, df.state.shift(periods=1) == 1), 1, 0)

In [7]:
# Trade list.
df1 = df[df.entry == 1]
df1['volatility'] = abs((df1.mid - df1.close) / df1.close)
# td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
# td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

<ipython-input-7-23b943c9db9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['volatility'] = abs((df1.mid - df1.close) / df1.close)
